In [ ]:
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tomllib
from common import plot_heatmap, to_matrix, transform

with open("plotting_config.toml", "rb") as fp:
    config = tomllib.load(fp)

In [ ]:
activity = pd.read_csv("../data/yjmob100k/activity.csv")
activity["count_log"] = activity["count"].apply(np.log10).fillna(0)

In [6]:
thresholds = [25, 50, 75] + list(range(100, 1000, 100)) + list(range(1000, 5001, 1000))

In [31]:
Path("../figures/threshold/").mkdir(parents=True, exist_ok=True)
for threshold in thresholds:
    activity["cut"] = activity["count"].apply(lambda x: 0 if x < threshold else 1)
    activity_raster_bin = to_matrix(activity, value="cut")
    activity_raster_bin = transform(activity_raster_bin)

    f_b, _ = plot_heatmap(activity_raster_bin, figsize=(200, 200), dpi=1, cmap="binary")
    f_b.savefig(
        f"../figures/threshold/activity_cut_{threshold}.png",
        facecolor="white",
        pad_inches=0,
        metadata=config["metadata"]["png"],
    )
    plt.close(f_b)   
    f_b, _ = plot_heatmap(activity_raster_bin, figsize=(4, 4), dpi=100, cmap="binary")
    f_b.savefig(
        f"../figures/threshold/activity_cut_400_{threshold}.png",
        facecolor="white",
        pad_inches=0,
        metadata=config["metadata"]["png"],
    )
    plt.close(f_b)
    f, a = plt.subplots(figsize=(4, 2.5), dpi=100, layout="constrained")
    sns.histplot(activity["count"], log_scale=(True, False), color="#81a1c1", ec="#2e3440", ax=a)
    a.axvline(threshold, color="#bf616a", lw=4)
    a.set_xlim([1, 40000])
    for i in ["svg", "png"]:
        f.savefig(
            f"../figures/threshold/threshold_{threshold}.{i}",
            facecolor="none",
            pad_inches=0,
            metadata=config["metadata"][i],
        )
    plt.close(f)